In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
users = pd.read_csv('users_info.csv', index_col=0)
users = users.rename(columns = {'users':'nickname'})

In [3]:
df_stocks = pd.read_csv('stocks_data/stocks_panel_data.csv')
df_stocks['Date'] = pd.to_datetime(df_stocks['Date'])
df_stocks['post utility'] = [0 for i in range(len(df_stocks))]
df_stocks

,Open,Adj Close,High,Low,Volume,Date,RSI,MACD,Symbols,post utility
0,5.1000,5.5340,5.66300,5.100000,3.372190e+07,2022-03-24,NaN,0.000000,CBOM,0
1,0.0181,0.0190,0.02177,0.015995,1.303864e+11,2022-03-24,NaN,0.000000,VTBR,0
2,251.4100,258.5100,278.90000,232.000000,8.587012e+07,2022-03-24,NaN,0.000000,GAZP,0
3,130.0000,135.4000,157.95000,130.000000,1.806938e+07,2022-03-24,NaN,0.000000,SBERP,0
4,0.6600,0.6815,0.69800,0.660000,6.837960e+08,2022-03-24,NaN,0.000000,HYDR,0
...,...,...,...,...,...,...,...,...,...,...
27169,18.9100,19.0200,19.02500,18.880000,4.663500e+06,2024-03-26,44.820717,0.427940,SVCB,0
27170,15156.0000,15130.0000,15314.00000,15106.000000,3.110100e+04,2024-03-26,57.348243,-76.325582,GMKN,0
27171,79.2000,79.4000,79.55000,78.550000,2.497600e+05,2024-03-26,47.315436,1.596217,RTKMP,0
27172,308.2000,307.1100,308.74000,304.800000,1.992500e+05,2024-03-26,62.002540,5.522067,BSPB,0


In [4]:
def calculate_post_utility(row):
    base_utility = np.sqrt(abs(row['dostoevski_score'] + row['bert_score']) / 2 * row['yields'] * row['followers'])
    if row['dostoevski_score'] + row['bert_score'] < 0:
        return -base_utility
    return base_utility

for file in os.listdir('text_data/'):
    df = pd.read_csv(f'text_data/{file}').drop(columns=['id', 'text']).rename(columns={'inserted': 'Date'})
    df['Date'] = pd.to_datetime(df['Date'])
    merged_df = df.merge(users, on='nickname', how='left')
    merged_df['yields'] = merged_df['yields'].clip(lower=0)
    merged_df['post utility'] = merged_df.apply(calculate_post_utility, axis=1)
    res = merged_df.groupby('Date')['post utility'].sum().reset_index()
    
    tick = file.split('_')[-1].split('.')[0]
    df_stocks_temp = df_stocks[df_stocks['Symbols'] == tick].copy()
    
    df_stocks_temp = df_stocks_temp.merge(res, on='Date', how='left', suffixes=('', '_updated'))
    df_stocks_temp['post utility'] = df_stocks_temp['post utility_updated'].fillna(df_stocks_temp['post utility'])
    df_stocks_temp.drop(columns=['post utility_updated'], inplace=True)
    
    df_stocks = pd.concat([df_stocks[df_stocks['Symbols'] != tick], df_stocks_temp])

df_stocks = df_stocks.sort_values(by=['Symbols', 'Date']).reset_index(drop=True)

In [5]:
df_stocks.fillna(0, inplace=True)
df_stocks

,Open,Adj Close,High,Low,Volume,Date,RSI,MACD,Symbols,post utility
0,11.505,12.890,14.414,11.500,93857400.0,2022-03-24,0.000000,0.000000,AFKS,0.000000
1,13.100,12.603,13.299,12.603,31222800.0,2022-03-25,0.000000,-0.022895,AFKS,-390.385590
2,12.661,11.650,12.698,11.400,20587500.0,2022-03-28,0.000000,-0.116594,AFKS,121.622148
3,11.650,11.540,12.400,11.300,31094600.0,2022-03-29,0.000000,-0.197451,AFKS,-2341.191734
4,12.200,12.500,12.690,11.710,27744600.0,2022-03-30,0.000000,-0.181970,AFKS,-1943.456041
...,...,...,...,...,...,...,...,...,...,...
27169,3817.800,3948.000,3972.600,3810.000,3671350.0,2024-03-20,76.833615,162.677363,YNDX,-4451.427133
27170,3952.000,3882.000,4004.800,3842.400,3389849.0,2024-03-21,72.177778,168.820742,YNDX,0.000000
27171,3885.000,3834.000,3944.600,3800.000,2441164.0,2024-03-22,64.489475,167.880987,YNDX,0.000000
27172,3850.000,4024.600,4025.000,3845.200,1938567.0,2024-03-25,65.487572,180.436097,YNDX,0.000000


In [6]:
df_stocks.to_csv('stocks_data/stocks_panel_data_with_utlity.csv')